# Multiagent system using GPT assistant in indistrial control case with SocietyOfMindAgent

This notebook demonstrates the SocietyOfMindAgent, which runs a group chat as an internal monologue, but appears to the external world as a single agent, which provides three distinct advantages:

1. It provides a clean way of producing a hierarchy of agents, hiding complexity as inner monologues.
2. It provides a consistent way of extracting an answer from a lengthy group chat (normally, it is not clear which message is the final response, and the response itself may not always be formatted in a way that makes sense when extracted as a standalone message).
3. It provides a way of recovering when agents exceed their context window constraints (the inner monologue is protected by try-catch blocks)

In [14]:
import autogen  # noqa: E402

config_list = autogen.config_list_from_json(
    "./OAI_CONFIG_LIST.json"  # All needed OpenAI info is in this file
)

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": config_list,
    "temperature": 0,
}


### Example Group Chat with Two Agents

In this example, we will use an AssistantAgent and a UserProxy agent (configured for code execution) to work together to solve a problem. Executing code requires *at least* two conversation turns (one to write the code, and one to execute the code). If the code fails, or needs further refinement, then additional turns may also be needed. We will then wrap these agents in a SocietyOfMindAgent, hiding the internal discussion from other agents (though will still appear in the console), and ensuring that the response is suitable as a standalone message.

#### Construct the Inner-Monologue Agents
We begin by constructing the inner-monologue agents. These are the agents that do that real work.

In [15]:
# GPT-3 assistant
gpt_assistant = autogen.AssistantAgent(
    "inner-assistant",
    system_message="You are a data analyst who help people find information through mathematic or statistics analyses.",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

# Statistics control process (SCP) caller
scp_caller = autogen.UserProxyAgent(
    "inner-scp-caller",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = autogen.GroupChat(
    agents=[gpt_assistant, scp_caller],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
)

#### Construct and Run the SocietyOfMind Agent
We now wrap the inner group-chat with the SocietyOfMind Agent, and create a UserProxy to talk to it.

In [16]:
from autogen.agentchat.contrib.society_of_mind_agent import SocietyOfMindAgent  # noqa: E402

task = '''
Given a dataset:      
time	pH  
0	2023-01-01 00:00:00	7.0  
1	2023-01-01 01:00:00	6.8  
2	2023-01-01 02:00:00	6.5  
3	2023-01-01 03:00:00	6.2  
4	2023-01-01 04:00:00	6.0  
5	2023-01-01 05:00:00	5.8  
6	2023-01-01 06:00:00	5.5  
7	2023-01-01 07:00:00	5.3  
8	2023-01-01 08:00:00	5.0  
9	2023-01-01 09:00:00	4.8.   
Step 1: Please calculate the mean_pH, and std_pH. 
Step 2: Then calculate the lower band which is defined as lower_band = mean_ph - 2.0 * std.  
Step 3: Once having the lower_band, please calculate the exact time the pH drop to the lower_band, if it did drop.
'''

society_of_mind_agent = SocietyOfMindAgent(
    "society_of_mind",
    chat_manager=manager,
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)


In [17]:

user_proxy.initiate_chat(society_of_mind_agent, message=task)

user_proxy (to society_of_mind):


Given a dataset:      
time	pH  
0	2023-01-01 00:00:00	7.0  
1	2023-01-01 01:00:00	6.8  
2	2023-01-01 02:00:00	6.5  
3	2023-01-01 03:00:00	6.2  
4	2023-01-01 04:00:00	6.0  
5	2023-01-01 05:00:00	5.8  
6	2023-01-01 06:00:00	5.5  
7	2023-01-01 07:00:00	5.3  
8	2023-01-01 08:00:00	5.0  
9	2023-01-01 09:00:00	4.8.   
Step 1: Please calculate the mean_pH, and std_pH. 
Step 2: Then calculate the lower band which is defined as lower_band = mean_ph - 2.0 * std.  
Step 3: Once having the lower_band, please calculate the exact time the pH drop to the lower_band, if it did drop.


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
society_of_mind (to chat_manager):


Given a dataset:      
time	pH  
0	2023-01-01 00:00:00	7.0  
1	2023-01-01 01:00:00	6.8  
2	2023-01-01 02:00:00	6.5  
3	2023-01-01 03:00:00	6.2  
4	2023-01-01 04:00:00	6.0  
5	2023-01-01 05:00:00	5.8  
6	2023-01-01 06:00:00	5.5  
7	2023-01-0

ChatResult(chat_id=None, chat_history=[{'content': '\nGiven a dataset:      \ntime\tpH  \n0\t2023-01-01 00:00:00\t7.0  \n1\t2023-01-01 01:00:00\t6.8  \n2\t2023-01-01 02:00:00\t6.5  \n3\t2023-01-01 03:00:00\t6.2  \n4\t2023-01-01 04:00:00\t6.0  \n5\t2023-01-01 05:00:00\t5.8  \n6\t2023-01-01 06:00:00\t5.5  \n7\t2023-01-01 07:00:00\t5.3  \n8\t2023-01-01 08:00:00\t5.0  \n9\t2023-01-01 09:00:00\t4.8.   \nStep 1: Please calculate the mean_pH, and std_pH. \nStep 2: Then calculate the lower band which is defined as lower_band = mean_ph - 2.0 * std.  \nStep 3: Once having the lower_band, please calculate the exact time the pH drop to the lower_band, if it did drop.\n', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'The mean_pH of the dataset is 5.89 and the std_pH is 0.7475. The lower_band, calculated as mean_pH - 2.0 * std_pH, is 4.3949. However, the exact time the pH dropped to the lower_band could not be determined based on the given dataset.', 'role': 'user', 'name': 'society_of_mi